In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv(r'dataset.csv')
data.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Brand,Model
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.60,998.0,58.16,5.0,1.75,Maruti,Wagon
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,12.50,Hyundai,Creta
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.20,1199.0,88.70,5.0,4.50,Honda,Jazz
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,6.00,Maruti,Ertiga
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,17.74,Audi,A4


In [3]:
X, y = data.drop(columns = ['Price','Name']),data['Price']

In [4]:
X.head()

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Brand,Model
0,Mumbai,2010,72000,CNG,Manual,First,26.60,998.0,58.16,5.0,Maruti,Wagon
1,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,Hyundai,Creta
2,Chennai,2011,46000,Petrol,Manual,First,18.20,1199.0,88.70,5.0,Honda,Jazz
3,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,Maruti,Ertiga
4,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,Audi,A4


In [5]:
y.head()

0     1.75
1    12.50
2     4.50
3     6.00
4    17.74
Name: Price, dtype: float64

In [6]:
X = pd.get_dummies(X)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [9]:
xg_reg = xgb.XGBRegressor(learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective='reg:linear',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

In [10]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_train)

C:\Users\acer\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[17:08:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [11]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_train, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.607867


In [12]:
print(y_train, preds)

4248     1.95
4129     7.52
2534     8.75
2893     6.95
2860     6.00
        ...  
3772     6.75
5191    32.90
5226     9.00
5390     4.95
860      5.50
Name: Price, Length: 4815, dtype: float64 [1.7512103 6.813821  8.919698  ... 9.133183  4.5885706 6.4612   ]


In [13]:
import matplotlib.pyplot as plt

xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [30,30]
plt.show()

<Figure size 640x480 with 1 Axes>

In [14]:
from sklearn.metrics import r2_score
r2_score(y_train, preds)

0.9970596662315073

In [15]:
from sklearn.model_selection import GridSearchCV

param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}

gsearch1 = GridSearchCV(estimator = xg_reg,
                        param_grid = param_test1, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)

gsearch1.fit(X_train,y_train)

gsearch1.cv_results_

C:\Users\acer\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[17:16:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


{'mean_fit_time': array([17.48998427, 16.44021297, 16.36381621, 27.07111216, 26.64466734,
        25.57524376, 41.30544782, 39.49997606, 37.19053822, 49.05755134,
        45.13422542, 39.16158471]),
 'std_fit_time': array([0.70908017, 0.38007924, 0.24869176, 0.72944206, 0.22641256,
        0.36447725, 0.85130157, 0.15445081, 1.44797573, 2.46082174,
        1.20110898, 4.00769907]),
 'mean_score_time': array([0.01815114, 0.01954756, 0.02353654, 0.02273936, 0.02074451,
        0.02553201, 0.03311162, 0.03637362, 0.03204737, 0.03408008,
        0.03111258, 0.03878689]),
 'std_score_time': array([0.00230877, 0.00232645, 0.00348895, 0.00477893, 0.00336137,
        0.00701248, 0.00482039, 0.0069468 , 0.0103628 , 0.004094  ,
        0.00189888, 0.00700612]),
 'param_max_depth': masked_array(data=[3, 3, 3, 5, 5, 5, 7, 7, 7, 9, 9, 9],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False],
        fill_value='?',
             

In [16]:
xg_reg1 = xgb.XGBRegressor(learning_rate =0.01,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=3,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective='reg:linear',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

In [17]:
xg_reg1.fit(X_train,y_train)

pred2 = xg_reg.predict(X_train)

[17:16:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [18]:
rmse = np.sqrt(mean_squared_error(y_train, pred2))
print("RMSE: %f" % (rmse))

RMSE: 0.607867


In [19]:
r2_score(y_train,pred2)

0.9970596662315073

In [20]:
pred_test = xg_reg.predict(X_test)

In [22]:
r2_score(y_test,pred_test)

0.9200842600483072

In [23]:
rmse = np.sqrt(mean_squared_error(y_test,pred_test))
print("RMSE: %f" % (rmse))

RMSE: 3.135983
